In [ ]:
# https://www.youtube.com/watch?v=j-3vuBynnOE
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
import tensorflow as tf
import numpy as np
import os

from keras.preprocessing.image import ImageDataGenerator

DATADIR = "C:\\Users\\tomto\\Desktop\\hand2\\data\\"
CATEGORIES = ["NONE", "ONE", "TWO", "THREE", "FOUR", "FIVE"]
IMG_SIZE = 256

for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)
        plt.imshow(img_array, cmap='gray')
        plt.show()

        break
    break

In [ ]:
print(category)

In [ ]:
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)

        for img in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE) 
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) 
                training_data.append([new_array, class_num]) 
            except Exception as e:  
                pass


create_training_data()

print(len(training_data))

In [ ]:
import random

random.shuffle(training_data)

X = []
y = []
for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = np.array(y)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
values = np.array(y)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = y.reshape(len(y), 1)
y = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = \
    train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

In [ ]:
# https://keras-cn.readthedocs.io/en/latest/preprocessing/image/
nbatch=32
datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=12,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             zoom_range=0.15,
                             horizontal_flip=True)
train_gen = datagen.flow(X_train, y_train, batch_size=32)
validation_gen = datagen.flow(X_val, y_val)
test_gen = datagen.flow(X_test, y_test)

In [ ]:
for X,y in test_gen:
    print(X.shape, y.shape)
    
    plt.figure(figsize=(16,16))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.axis('off')
        plt.title('Label: %d' % np.argmax(y[i]))
        img = np.uint8(255*X[i,:,:,0])
        plt.imshow(img)
    break

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(256,256,1)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(150,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(6,activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
callbacks_list=[EarlyStopping(monitor='val_loss', patience=10),
               ModelCheckpoint(filepath='hand1model.h6',
                               monitor='val_loss',
                               save_best_only=True),]

In [ ]:
# model.save("model/")
os.environ["CUDA_VISIBLE_DEVICES"]="0"

with tf.device('/GPU:0'):
    history = model.fit_generator(
        train_gen,
        steps_per_epoch=64,
        epochs=200,
        validation_data=validation_gen,
        validation_steps=28,
        callbacks=callbacks_list
    )

In [ ]:
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
nepochs=len(history.history['loss'])

plt.plot(range(nepochs), history.history['loss'],'g-', label='train') 
plt.plot (range(nepochs), history.history['val_loss'], 'c-', label='test')

plt.legend(prop={'size': 20})



plt.ylabel('loss')
plt.xlabel('number of epochs')
plt.subplot(1,2,2)
plt.plot(range(nepochs), history.history['acc'], label='train')
plt.plot(range(nepochs), history.history['val_acc'], 'c-', label='test')
plt.legend(prop={'size': 20})
plt.ylabel('accuracy')
plt.xlabel('number of epochs')

In [ ]:
model.save('idhand.h6')

In [ ]:
y_pred = np.int32([np.argmax(r) for r in model.predict(X_test)])
y_test = np.int32([np.argmax(r) for r in y_test])
match = (y_test == y_pred)
print(np.sum(match)*100/match.shape[0])

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [ ]:
from keras.models import load_model
import cv2

model = load_model('handmodel.h6')

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

img = cv2.imread('C:\\Users\\tomto\\Desktop\\hand1\\data\\FIVE\\FIVE_13.png')
img = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
img = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

classes = model.predict_classes(img)
print(img)
print(classes)
